# RETICOLO con passo 300

$d\sin\theta = n\lambda$ con n numero intero

In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
from jax import grad
from jax import numpy as jnp

dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')

from giopy import Interpolazione, final_val

sys.path.append(dir_path)
from spectra_reppresentation import draw_spectrum

dir_path = os.path.abspath('')
FILE = dir_path + '/data/' + 'reticolo_300.csv'
fr = pd.read_csv(FILE)
fr = fr.set_index('colore')

In [18]:
def formatter(data) -> dict:
    keys = ['n1','n2','n3','-n1','-n2','-n3']
    d = {}
    for i in range(0,len(data),2): # i = gradi, i+1 = primi
        d[keys[int(i/2)]] = np.float64(data[i] + data[i+1]/60)
    return d

giallo = formatter(fr.loc['GIALLO']) # come argomento passo una la riga
verde = formatter(fr.loc['VERDE']) # NOTE: i dati raccolti hanno un'offset di 90 gradi, la correzione è applicata in formatter
acqua = formatter(fr.loc['ACQUA'])

λ = {'GIALLO':590,'VERDE':569,'ACQUA':498}
sigmaθ = np.radians(0.044)  # deg

$d = \frac{n\lambda}{\sin\theta}$

In [19]:
def passo(n,λ,θ):
    return n*λ/jnp.abs(jnp.sin(jnp.radians(θ)))

def sigma_passo(n,λ,θ):
    dddθ = np.array([grad(passo,argnums=2)(i,λ,g) for i,g in zip(n,θ)])
    return jnp.abs(dddθ * sigmaθ)

m = np.array([1,2,3,1,2,3],dtype=np.int16)

d_giallo = passo(m,λ['GIALLO'],np.array(list(giallo.values()))) # nm
sigmad_giallo = sigma_passo(m,λ['GIALLO'],np.array(list(giallo.values()))) # nm

d_verde = passo(m,λ['VERDE'],np.array(list(verde.values()))) # nm
sigmad_verde = sigma_passo(m,λ['VERDE'],np.array(list(verde.values()))) # nm

d_acqua = passo(m,λ['ACQUA'],np.array(list(acqua.values()))) # nm
sigmad_acqua = sigma_passo(m,λ['ACQUA'],np.array(list(acqua.values()))) # nm

# print(d_giallo,d_verde,d_acqua)
d = np.concatenate((d_giallo,d_verde,d_acqua)) * 10**-9 # m
sigmad = np.concatenate((sigmad_giallo,sigmad_verde,sigmad_acqua)) * 10**-9 # m

dm = np.mean(d)
dstd = np.std(d)
print('Passo: ',final_val(dm,dstd,2,-6,'m'))
print('1/300 reticoli/mm',np.round(1/300 * 1000,2))

Passo:  (3.29 ± 0.01)e-6 m
1/300 reticoli/mm 3.33


# DETERMINAZIONE GAS INCOGNITO:

In [20]:
FILE = dir_path + '/data/' + 'reticolo_300_ignoto.csv'
fr = pd.read_csv(FILE)
fr = fr.set_index('colore')

In [26]:
giallo = formatter(fr.loc['GIALLO'][:6]) # come argomento passo una la riga
verde = formatter(fr.loc['VERDE'][:6]) # NOTE: i dati raccolti hanno un'offset di 90 gradi, la correzione è applicata in formatter
acqua = formatter(fr.loc['ACQUA'][:6])
blu = formatter(fr.loc['BLU'][:6])

m = np.array([1,2,3],dtype=np.int16)
d,sigmad = dm,dstd # rinomino variabili
#sigma teta

In [27]:
def lambda_val(n,θ,d):
    return n*d/jnp.sin(jnp.radians(θ))

def sigma_lambda(n,θ,d):
    dddd = np.array([grad(lambda_val,argnums=2)(i,g,d) for i,g,d in zip(n,θ,d)])
    dddteta = np.array([grad(lambda_val,argnums=1)(i,g,d) for i,g,d in zip(n,θ,d)])
    return jnp.sqrt(dddd**2 * sigmad**2 + dddteta**2 * sigmaθ**2)

λ = lambda_val(m,np.array(list(giallo.values())),d) # nm
sigma_lambda = sigma_lambda(m,np.array(list(giallo.values())),d) # nm

draw_spectrum(λ,sigma_lambda)

TypeError: 'numpy.float64' object is not iterable